# Day 21

link: https://adventofcode.com/2022/day/21

สำหรับข้อนี้ ลิงแต่ละตัวก็คือ expression ซึ่งอาจจะเป็น constant หรือเป็น binary operation จาก expression อื่น
อาจจะเขียนเป็น grammar ได้เลยว่า `expr := const | expr op expr`

เราเก็บ input ลง Hash โดยใช้ชื่อลิงเป็น key โดย parse constant term เป็นตัวเลข และเก็บ operation เป็น symbol

In [1]:
input = IO.foreach('in21.txt').to_a.map(&:strip)

monkeys = {}

input.each{|line|
  key, val = line.split(': ')
  args = val.split(' ')
  if args.size == 1
    args[0] = args[0].to_i
  else
    args[1] = args[1].to_sym
  end
  monkeys[key] = args
}
nil

## Part 1

part แรกเราก็แค่ evaluate expression แบบ recursive โดยเริ่มจาก "root"

เราอาจจะเก็บ result ของแต่ละ sub-expression ลง cache ไว้ด้วยก็ได้ เผื่อมีการใช้ซ้ำ แต่พอเช็คดูแล้วเราไม่พบการใช้ซ้ำเลย
ก็คือ evaluation tree มันเป็น tree จริงๆ เพราะฉะนั้นไม่จำเป็นต้อง cache

In [2]:
def evaluate(monkeys, monkey)
  v = monkeys[monkey]
  if v.size == 1
    v[0]
  else
    l = evaluate(monkeys, v[0])
    r = evaluate(monkeys, v[2])
    l.send(v[1], r)
  end
end

puts evaluate(monkeys, 'root')

158661812617812


## Part 2

part หลัง เรามอง "humn" เป็นตัวแปร x เมื่อเอาไปบวก ลบ หรือคูณกับ constant เราก็จะได้ expression เป็น polynomial ดีกรี 1: Ax + B โดย A, B เป็น rational number

เราเช็คมาแล้วว่าไม่มีการใช้ expression ซ้ำ ดังนั้น degree มันจะไม่โตไปกว่า 1 แน่นอน

การหารซับซ้อนกว่า ถ้าเราเอา polynomial / constant เราก็ยังได้ degree-1 polynomial เหมือนเดิม
แต่ถ้าเป็น constant / polynomial มันจะกลายเป็น degree-1 rational function แทน: $\frac{Ax + B}{Cx+D}$

เพื่อความง่ายเรา assume ว่าจะไม่มีการหารด้วย polynomial โชคดีที่เป็นจริงตามนั้น

โจทย์บอกให้เราเปลี่ยน root ให้เป็น x = y ซึ่งเทียบเท่ากับ x - y = 0 พอเรา evaluate expression ทั้งหมด ก็จะได้ polynomial Ax+B = 0
ดังนั้น x = -B/A

In [3]:
def evaluate_poly1(monkeys, monkey)
  v = monkeys[monkey]
  if monkey == 'humn'
    [Rational(1), Rational(0)]
  elsif v.size == 1
    [Rational(0), Rational(v[0])]
  else
    l1, l0 = evaluate_poly1(monkeys, v[0])
    r1, r0 = evaluate_poly1(monkeys, v[2])
    
    if v[1] == :- || monkey == 'root'
      [l1 - r1, l0 - r0]
    elsif v[1] == :+
      [l1 + r1, l0 + r0]
    elsif v[1] == :*
      [l1 * r0 + r1 * l0, r0 * l0]
    else
      raise 'divided by non-constant' if r1 != 0
      [l1 / r0, l0 / r0]
    end
  end
end

a, b = evaluate_poly1(monkeys, 'root')
puts (-b / a).to_i

3352886133831
